In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from tqdm import tqdm
from Base.DataIO import DataIO

#----Recommenders----
from SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from SLIM.SlimElasticNet import SLIMElasticNetRecommender
from cf.item_cf3 import ItemBasedCollaborativeFiltering
from cf.user_cf2 import UserBasedCollaborativeFiltering
from MF.ALS import AlternatingLeastSquare
from cbf.cbf import ContentBasedFiltering
from SlimBPR.SlimBPRRec import SlimBPRRec
from SlimBPR.SlimBPR import SlimBPR

from Hybrid.hybridRec import HybridRecommender
#---------------

#----Model Load & Save----
from Data_manager.Dataset import Dataset
#-------------------------

from sklearn.model_selection import train_test_split

**Dataset loading with pandas**

The function read_csv from pandas provides a wonderful and fast interface to load tabular data like this. For better results and performance we provide the separator ::, the column names ["user_id", "item_id", "ratings", "timestamp"], and the types of each attribute in the dtype parameter.

In [ ]:
def load_data():
  return pd.read_csv("./data_train.csv")

In [ ]:
ratings=load_data()
d ={'user_id': ratings['row'],'item_id':ratings['col'],'ratings':ratings['data']}
ratings=pd.DataFrame(data=d)

In [ ]:
ratings.dtypes

In [ ]:
userList=list(d['user_id'])
itemList=list(d['item_id'])
ratingList=list(d['ratings'])

In [ ]:
URM = sp.coo_matrix((ratingList,(userList,itemList)))
URM = URM.tocsr()

In [ ]:
URM

In [2]:
def load_data_ICM():
  return pd.read_csv("./data_ICM_title_abstract.csv")

In [3]:
features=load_data_ICM()
d ={'item_id': features['row'],'feature_id':features['col'],'value':features['data']}
features=pd.DataFrame(data=d)
itemList=list(d['item_id'])

In [4]:
featureList=list(d['feature_id'])

In [5]:
valueList=list(d['value'])
ICM = sp.coo_matrix((valueList,(itemList,featureList)))
ICM = ICM.tocsr()

In [6]:
ICM

<25975x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 490691 stored elements in Compressed Sparse Row format>

In [ ]:
num_users=URM.shape[0]

In [ ]:
num_items=URM.shape[1]

In [ ]:
num_users,num_items

**Dataset splitting into train,validation and test**

This is the last part before creating the recommender. However, this step is super important, as it is the base for the training, parameters optimization, and evaluation of the recommender(s).

In here we read the ratings (which we loaded and preprocessed before) and create the train, validation, and test User-Rating Matrices (URM). It's important that these are disjoint to avoid information leakage from the train into the validation/test set, in our case, we are safe to use the train_test_split function from scikit-learn as the dataset only contains one datapoint for every (user,item) pair. On another topic, we first create the test set and then we create the validation by splitting again the train set.

train_test_split takes an array (or several arrays) and divides it into train and test according to a given size (in our case testing_percentage and validation_percentage, which need to be a float between 0 and 1).

After we have our different splits, we create the sparse URMs by using the csr_matrix function from scipy.




In [ ]:
from Notebooks_utils.data_splitter import train_test_holdout

urm_train_validation, urm_test = train_test_holdout(URM, train_perc = 0.85)
urm_train, urm_validation = train_test_holdout(urm_train_validation, train_perc = 0.85)

In [ ]:
#Save Data
dataset = Dataset("prova", {"urm_train" : urm_train}, {"urm_validation" : urm_validation}, {"urm_test" : urm_test}) 
dataset.save_data("Salvataggi/")

In [7]:
#Load
loaded_dataset = Dataset(None, None, None, None)
loaded_dataset.load_data("Salvataggi/")


urm_train = (loaded_dataset.get_urm_train())["urm_train"]
urm_validation = (loaded_dataset.get_urm_validation())["urm_validation"]
urm_test = loaded_dataset.get_urm_test()["urm_test"]

In [8]:
urm_train_validation = urm_train + urm_validation

**Hyper parameter TUNING**

In [ ]:
from Base.Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

**Tuning User and Item CF**

In [ ]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["knn"] = Integer(5, 1000)
hyperparameters_range_dictionary["shrink"] = Integer(0, 1000)
hyperparameters_range_dictionary["similarity"] = Categorical(["cosine",
#                                                              "pearson" 
#                                                               "adjusted",
#                                                               "asymmetric",
                                                               "jaccard",
#                                                               "tanimoto", 
#                                                               "dice", 
#                                                               "tversky"
                                                             ])

In [ ]:
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = UserBasedCollaborativeFiltering

parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)

In [ ]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 50
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"

In [ ]:
parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize,
                      )

In [ ]:
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
best_parameters = search_metadata["hyperparameters_best"]
best_parameters
#last: 381, 515

**Tuning ALS**

In [ ]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["n_factors"] = Integer(20, 500)
hyperparameters_range_dictionary["regularization"] = Real(0.1, 2)
hyperparameters_range_dictionary["iterations"] = Integer(15, 150)

In [ ]:
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = AlternatingLeastSquare

parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)

In [ ]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 50
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"

In [ ]:
parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize,
                      )

**Evaluation Metrics**

In [9]:
def recall(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant) / relevant_items.shape[0]
    
    return recall_score
    
    
def precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant) / recommendations.shape[0]

    return precision_score

def mean_average_precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(precision_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

**Evaluation Procedure**

The evaluation procedure returns the averaged accuracy scores (in terms of precision, recall and MAP) for all users (that have at least 1 rating in the test set). It also calculates the number of evaluated and skipped users. It receives a recommender instance, and the train and test URMs.

In [10]:
def evaluator(recommender: object, urm_train: sp.csr_matrix, urm_test: sp.csr_matrix):
    recommendation_length = 10
    accum_precision = 0
    accum_recall = 0
    accum_map = 0
    
    num_users = urm_train.shape[0]
    
    num_users_evaluated = 0
    num_users_skipped = 0
    
    for user_id in range(num_users):
        user_profile_start = urm_test.indptr[user_id]
        user_profile_end = urm_test.indptr[user_id+1]

        relevant_items = urm_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
            
#         recommendations = recommender.recommend(user_id_array=user_id,
#                                                cutoff=recommendation_length,
#                                                remove_seen_flag=True
#                                                )

        expected_ratings = recommender.get_expected_ratings(user_id)
        recommended_items = np.flip(np.argsort(expected_ratings), 0)

        unseen_items_mask = np.in1d(recommended_items,urm_train[user_id].indices,
                                        assume_unique=True, invert=True)

        recommendations = recommended_items[unseen_items_mask]


        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1

    
    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)
    
    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

In [11]:
#Params for the recommenders
cbf_param = {
    "knn": 380,
    "shrink": 9
}

user_cf_param = {
    "knn": 249,
    "shrink": 853
}

item_cf_param = {
    "knn": 119,
    "shrink": 430
}

slim_bpr_param = {
    "learning_rate" : 0.05,
    "epochs": 10,
    "nnz" : 1,
    "knn": 200
}

als_param = {
    "n_factors": 213,
    "regularization": 0.11348942863855206,
    "iterations": 15
}


In [12]:
userCF = UserBasedCollaborativeFiltering(urm_train_validation)

In [13]:
userCF.fit(knn=user_cf_param["knn"], shrink=user_cf_param["shrink"],similarity='cosine')

Similarity column 25975 ( 100 % ), 2197.47 column/sec, elapsed time 0.20 min


In [14]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(userCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.05646164301780437, 5007, 2940)

In [15]:
itemCF = ItemBasedCollaborativeFiltering(urm_train_validation)

In [16]:
itemCF.fit(knn=item_cf_param["knn"], shrink=item_cf_param["shrink"], similarity="cosine")

Similarity column 25975 ( 100 % ), 2187.59 column/sec, elapsed time 0.20 min


In [17]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.05646850686616334, 5007, 2940)

In [ ]:
SlimBpr=SlimBPRRec(learning_rate=slim_bpr_param["learning_rate"], epochs=slim_bpr_param["epochs"], nnz=slim_bpr_param["nnz"], knn=slim_bpr_param["knn"])

In [ ]:
SlimBpr.fit(urm_train)

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(SlimBpr, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
slim_elastic = SLIMElasticNetRecommender(urm_train)

In [ ]:
slim_elastic.fit()

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(slim_elastic, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [18]:
cbf = ContentBasedFiltering(urm_train_validation,ICM)

In [19]:
cbf.fit(knn=cbf_param["knn"],shrink=cbf_param["shrink"],similarity='cosine')

Similarity column 25975 ( 100 % ), 974.77 column/sec, elapsed time 0.44 min


In [20]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(cbf, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.030656486777541514, 5007, 2940)

In [21]:
ALS = AlternatingLeastSquare(urm_train_validation)

In [22]:
ALS.fit(n_factors=als_param["n_factors"], regularization=als_param["regularization"],iterations=als_param["iterations"])

In [23]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(ALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.06676325359264511, 5007, 2940)

**Hybrid**

**Hybrid Recommender Tuning**

In [ ]:
from Base.Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

In [ ]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["w_user"] = Integer(0, 1000)
hyperparameters_range_dictionary["w_item"] = Integer(0, 1000)
hyperparameters_range_dictionary["w_cbf"] = Integer(0, 300)
hyperparameters_range_dictionary["w_als"] = Integer(0, 1000)

In [ ]:
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = HybridRecommender

parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)

In [ ]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train, ICM, userCF, itemCF, cbf, ALS],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [user_cf_param, item_cf_param, cbf_param, slim_bpr_param, als_param],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [urm_train_validation, ICM, userCF, itemCF, cbf, ALS],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [user_cf_param, item_cf_param, cbf_param, slim_bpr_param, als_param],
    FIT_KEYWORD_ARGS = {}
)

In [ ]:
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 300
n_random_starts = int(n_cases*0.45)
metric_to_optimize = "MAP"

In [ ]:
parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize,
                      )

In [ ]:
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
best_parameters = search_metadata["hyperparameters_best"]
best_parameters

In [36]:
w = {
    "user_cf": 853,
    "item_cf": 460,
    "cbf": 44,
    "icm_svd": 0,
    "als": 642,
    "slim": 0,
    "elastic": 0
}

In [37]:
recommender = HybridRecommender(urm_train_validation, ICM, userCF, itemCF, cbf, ALS)

In [38]:
recommender.fit(user_cf_param=user_cf_param,item_cf_param=item_cf_param,cbf_param=cbf_param,
                slim_param=slim_bpr_param,als_param=als_param, w_user=w["user_cf"], w_item=w["item_cf"], w_cbf=w["cbf"], w_als=w["als"])

In [39]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommender, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.00013115158761398967, 1.0, 0.06974328914723055, 5007, 2940)

**Submission to competition**

This step serves as a similar step that you will perform when preparing a submission to the competition. Specially after you have chosen and trained your recommender.

For this step the best suggestion is to select the most-performing configuration obtained in the hyperparameter tuning step and to train the recommender using both the train and validation set. Remember that in the competition you do not have access to the test set.

Another consideration is that, due to easier and faster calculations, we replaced the user/item identifiers with new ones in the preprocessing step. For the competition, you are required to generate recommendations using the dataset's original identifiers. Due to this, this step also reverts back the newer identifiers with the ones originally found in the dataset.

Last, this step creates a function that writes the recommendations for each user in the same file in a tabular format following this format:

csv
<user_id>,<item_id_1> <item_id_2> <item_id_3> <item_id_4> <item_id_5> <item_id_6> <item_id_7> <item_id_8> <item_id_9> <item_id_10>
Always verify the competitions' submission file model as it might vary from the one we presented here.

In [28]:
def load_goodguys():
  return pd.read_csv("./data_target_users_test.csv")
goodguys=load_goodguys()

In [29]:
goodguys

,user_id
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [30]:
users_to_recommend = np.random.choice(goodguys.user_id,size=goodguys.size, replace=False)
users_to_recommend

array([1404, 5908, 1978, ..., 6821, 2000, 2714], dtype=int64)

In [31]:
def prepare_submission(users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object):
    
    recommendation_length = 10
    submission = []
    
    for user_id in users_to_recommend :

        recommendations = recommender.recommend2(user_id, urm_train, recommendation_length)

        
        submission.append((user_id, [item_id for item_id in recommendations]))
   
    return submission

In [32]:
submission = prepare_submission(users_to_recommend, urm_train_validation, recommender)

In [33]:
submission

[(1404, [5581, 18419, 15363, 7883, 10418, 19874, 14684, 1816, 14137, 15754]),
 (5908, [11231, 5254, 19489, 11093, 11283, 11694, 12241, 11175, 17736, 8588]),
 (1978, [14210, 10675, 7718, 6010, 23001, 25116, 9591, 21552, 22003, 9925]),
 (5942, [4825, 11925, 23109, 4150, 4222, 15144, 13686, 22196, 1464, 12632]),
 (667, [25631, 20095, 2665, 2236, 8097, 10035, 24900, 13294, 8990, 5803]),
 (1284, [6088, 9386, 12293, 9315, 16996, 13541, 10789, 24764, 19312, 5498]),
 (1141, [11197, 306, 21042, 25133, 24032, 625, 5930, 21395, 12943, 18924]),
 (3290, [3497, 4147, 588, 21061, 3744, 8986, 8337, 12486, 7642, 12992]),
 (7242, [17209, 9733, 4980, 11340, 13136, 16173, 4837, 6708, 5, 654]),
 (2649, [18851, 8860, 597, 22171, 20974, 22010, 3758, 18202, 3832, 12399]),
 (7281, [9555, 10594, 10834, 23154, 16928, 13711, 8097, 10981, 12818, 22848]),
 (6713, [6865, 24300, 25698, 19028, 388, 5491, 3418, 7117, 7835, 19317]),
 (5008, [10594, 2492, 23154, 10345, 9701, 5607, 5, 2426, 1583, 1619]),
 (2553, [2426, 15

In [34]:
import os
from datetime import datetime

csv_fname = './submission'
csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

def write_submission(submissions):
    with open(csv_fname, "w") as f:
        f.write(f"user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [35]:
write_submission(submission)